##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：提示影片版


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/Video.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


這個筆記本提供了關於如何使用影片檔案提示 Gemini 1.5 Pro 的快速範例。這個範例將使用 [Big Buck Bunny](https://peach.blender.org/about/) 的短片。


In [ ]:
!pip install -U google-generativeai

In [ ]:
import google.generativeai as genai

### 驗證概述

**重要事項：** 檔案 API 使用 API 金鑰進行驗證與存取。上傳的檔案會與 API 金鑰的雲端專案產生關聯。與其他使用 API 金鑰的 Gemini API 不同，你的 API 金鑰也會授予存取你已上傳至檔案 API 的資料的權限，因此請務必格外小心，保護你的 API 金鑰安全。有關保護 API 金鑰的最佳做法，請參閱 Google 的 [文件](https://support.google.com/googleapi/answer/6310037)。


### 設定你的 API 金鑰

如要執行以下單元格，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab Secret 中。如果你尚未擁有 API 金鑰，或不確定如何建立 Colab Secret，請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb)取得範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 將影片上傳至檔案 API

Gemini API 直接接受影片檔案格式。檔案 API 接受大小在 2 GB 以下的檔案，每個專案最多可儲存 20 GB 的檔案。檔案會保留 2 天，且無法透過 API 下載。在此範例中，你將使用影片「Big Buck Bunny」。

> 「Big Buck Bunny」為 (c) 2008 年，Blender Foundation / www.bigbuckbunny.org 所有，並在 [創用 CC 姓名標示 3.0](http://creativecommons.org/licenses/by/3.0/) 授權下，[取得授權](https://peach.blender.org/about/)。

注意：你也可以 [上傳自己的檔案](https://github.com/google-gemini/cookbook/tree/main/examples/Upload_files.ipynb) 來使用。


In [22]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

--2024-05-14 14:10:23--  https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4
Resolving download.blender.org (download.blender.org)... 104.22.65.163, 104.22.64.163, 172.67.14.163, ...
Connecting to download.blender.org (download.blender.org)|104.22.65.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64657027 (62M) [video/mp4]
Saving to: ‘BigBuckBunny_320x180.mp4.2’

BigBuckBunny_320x18 100%[===================>]  61.66M  39.0MB/s    in 1.6s    

2024-05-14 14:10:25 (39.0 MB/s) - ‘BigBuckBunny_320x180.mp4.2’ saved [64657027/64657027]



In [23]:
video_file_name = "BigBuckBunny_320x180.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/2kefwb3adzuv


## 取得檔案

上傳檔案後，你可以透過呼叫「files.get」來確認 API 已成功收到檔案。

「files.get」讓你可以看到上傳到 File API 的檔案，這些檔案與你的 API 金鑰所屬的 Cloud 專案相關聯。只有「名稱」(和其延伸屬性「URI」) 是唯一的。


In [24]:
import time

while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)
print(f'Video processing complete: ' + video_file.uri)

Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/2kefwb3adzuv


## 自動產生內容

影片上傳後，你可以建立參考檔案 API URI 的 `GenerateContent` 要求。


In [ ]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

Making LLM inference request...
The video begins with a panoramic view of a lush green field. A cloud with hues of pink and yellow glides across the sky as the camera pans to focus on a stream flowing through the field. The title card "Big Buck Bunny" appears next.

A large white rabbit, the protagonist, emerges from its burrow and stretches. It takes in the beautiful day, smelling flowers and enjoying the sunshine. The rabbit sees a red apple on the ground and excitedly picks it up. It walks towards a group of smaller animals: a chinchilla, a fox, and a squirrel, who are also enjoying the day. The rabbit wants to share the apple but the squirrel throws it away. The rabbit gets sad and starts to eat grass.

Seeing the rabbit's sadness, the squirrel comes up with a plan to tease it. The squirrel throws an apple at the rabbit's head. The rabbit gets angry and chases after the squirrel. They run around the tree, with the chinchilla and fox looking on. The squirrel then throws a spiky ball

## 刪除檔案

檔案會在 2 天後自動刪除，或是你可以用 `files.delete()` 手動刪除檔案。


In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')

Deleted file https://generativelanguage.googleapis.com/v1beta/files/2kefwb3adzuv


## 進一步了解

使用檔案 API，你可以上傳種類繁多的多模態 MIME 類型，包括圖片、聲音和影片格式。 檔案 API 可處理可用於使用 [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) 或 [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent) 產生內容的輸入。

檔案 API 接受大小小於 2 GB 的檔案，每個專案可以儲存的檔案總量上限為 20 GB。 檔案會保留 2 天，而且無法從 API 下載。

* 透過入門文件了解 [檔案 API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb)。

* 在文件了解更進一步的 [媒體檔案提示](https://ai.google.dev/tutorials/prompting_with_media)資訊，包括支援的格式和長度上限。
